In [1]:
%pip uninstall hopsworks hsfs hsml -y

%pip install hopsworks

Found existing installation: hopsworks 4.1.4
Uninstalling hopsworks-4.1.4:
  Successfully uninstalled hopsworks-4.1.4
Note: you may need to restart the kernel to use updated packages.
  Using cached hopsworks-4.1.4-py3-none-any.whl.metadata (11 kB)
Using cached hopsworks-4.1.4-py3-none-any.whl (640 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
from datetime import datetime, timedelta, timezone
import pandas as pd
from xgboost import XGBRegressor
import hopsworks
import json
import os

In [3]:
today = datetime.today().replace(hour=0, minute=0, second=0, microsecond=0)
today

datetime.datetime(2025, 1, 5, 0, 0)

In [4]:
project = hopsworks.login(project="dbillLab1")
fs = project.get_feature_store()

feature_view = fs.get_feature_view(
    name='tmdb_fv',
    version=1,
)

2025-01-05 18:11:08,720 INFO: Initializing external client
2025-01-05 18:11:08,720 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-05 18:11:10,687 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1169559


In [5]:
mr= project.get_model_registry()

retrieved_model=mr.get_model(
    name="movie_rating_xgboost_model",
    version=1,
)

saved_model_dir=retrieved_model.download()

In [63]:
retrieved_xgboost_model=XGBRegressor()

retrieved_xgboost_model.load_model(saved_model_dir+"/imdb_rating_model.pkl")

retrieved_xgboost_model

XGBoostError: [20:00:57] /Users/runner/work/xgboost/xgboost/src/common/io.cc:147: Opening /var/folders/0y/84b2kd2s4f16pkg8r__5ny5c0000gn/T/44f6ee2d-6387-40d7-8502-8583e39b8c23/movie_rating_xgboost_model/1/model.json failed: No such file or directory
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x0000000157944454 dmlc::LogMessageFatal::~LogMessageFatal() + 124
  [bt] (1) 2   libxgboost.dylib                    0x0000000157a2d36c xgboost::common::LoadSequentialFile(std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> >) + 728
  [bt] (2) 3   libxgboost.dylib                    0x00000001579c8270 XGBoosterLoadModel::$_4::operator()() const + 144
  [bt] (3) 4   libxgboost.dylib                    0x00000001579c7e70 XGBoosterLoadModel + 620
  [bt] (4) 5   libffi.8.dylib                      0x0000000102e2004c ffi_call_SYSV + 76
  [bt] (5) 6   libffi.8.dylib                      0x0000000102e1d834 ffi_call_int + 1404
  [bt] (6) 7   _ctypes.cpy

In [15]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("alanvourch/tmdb-movies-daily-updates")

# List files in the downloaded dataset directory
files = os.listdir(path)
print("Files in dataset directory:", files)

# Read the first file
updated_df = pd.read_csv(os.path.join(path, files[0]))

# Print the first 5 rows
updated_df.head()

Files in dataset directory: ['TMDB_all_movies.csv']


,id,title,vote_average,vote_count,status,release_date,revenue,runtime,budget,imdb_id,...,spoken_languages,cast,director,director_of_photography,writers,producers,music_composer,imdb_rating,imdb_votes,poster_path
0,2,Ariel,7.1,335.0,Released,1988-10-21,0.0,73.0,0.0,tt0094675,...,suomi,"Heikki Salomaa, Hanna Jokinen, Matti Pellonpää...",Aki Kaurismäki,Timo Salminen,Aki Kaurismäki,Aki Kaurismäki,NaN,7.4,8868.0,/ojDg0PGvs6R9xYFodRct2kdI6wC.jpg
1,3,Shadows in Paradise,7.3,400.0,Released,1986-10-17,0.0,74.0,0.0,tt0092149,...,"English, suomi, svenska","Teuvo Rissanen, Malla Hukkanen, Olli Varja, Ju...",Aki Kaurismäki,Timo Salminen,Aki Kaurismäki,Mika Kaurismäki,NaN,7.5,7646.0,/nj01hspawPof0mJmlgfjuLyJuRN.jpg
2,5,Four Rooms,5.9,2650.0,Released,1995-12-09,4257354.0,98.0,4000000.0,tt0113101,...,English,"Lili Taylor, Valeria Golino, Marisa Tomei, Mar...","Quentin Tarantino, Robert Rodriguez, Allison A...","Andrzej Sekula, Rodrigo García, Guillermo Nava...","Quentin Tarantino, Robert Rodriguez, Allison A...","Quentin Tarantino, Lawrence Bender, Alexandre ...",Combustible Edison,6.7,113064.0,/pyCk5JgtRZwRxnXwfrvyzukaKue.jpg
3,6,Judgment Night,6.5,333.0,Released,1993-10-15,12136938.0,109.0,21000000.0,tt0107286,...,English,"Stephen Dorff, Everlast, Will Zahrn, Emilio Es...",Stephen Hopkins,Peter Levy,"Jere Cunningham, Lewis Colick","Gene Levy, Marilyn Vance, Lloyd Segan",Alan Silvestri,6.6,19437.0,/3rvvpS9YPM5HB2f4HYiNiJVtdam.jpg
4,8,Life in Loops (A Megacities RMX),7.5,27.0,Released,2006-01-01,0.0,80.0,42000.0,tt0825671,...,"English, हिन्दी, 日本語, Pусский, Español",NaN,Timo Novotny,Wolfgang Thaler,"Michael Glawogger, Timo Novotny","Ulrich Gehmacher, Timo Novotny",NaN,8.2,284.0,/7ln81BRnPR2wqxuITZxEciCe1lc.jpg


In [16]:
updated_df['status'].value_counts()

status
Released           1021151
Planned               5278
Post Production       5189
In Production         5028
Rumored               1090
Canceled               596
Name: count, dtype: int64

In [28]:
df=updated_df.copy()

df_planned = df[df['status'] == 'Planned']  #5278

zero_count_B = (df_planned['budget'] == 0.0).sum()
print("zero budget", zero_count_B)

print(df_planned['imdb_votes'].isnull().sum())

df_planned.head()

zero revenue 5265
zero budget 4990
5259


,id,title,vote_average,vote_count,status,release_date,revenue,runtime,budget,imdb_id,...,spoken_languages,cast,director,director_of_photography,writers,producers,music_composer,imdb_rating,imdb_votes,poster_path
5604,10526,Cancer Vixen: A True Story,0.0,0.0,Planned,NaN,0.0,0.0,0.0,tt3118424,...,English,NaN,NaN,NaN,NaN,Tim Bevan,NaN,NaN,NaN,NaN
30240,45654,X: The Man with the X-Ray Eyes,0.0,0.0,Planned,NaN,0.0,0.0,0.0,tt1399133,...,NaN,NaN,Juan Carlos Fresnadillo,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39323,57208,Analog,0.0,0.0,Planned,NaN,0.0,0.0,0.0,tt1843097,...,NaN,NaN,Mike Million,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50733,72361,Thicker,10.0,1.0,Planned,NaN,0.0,0.0,1500000.0,tt0914385,...,NaN,NaN,Terry Nemeroff,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62404,87513,Motor City,0.0,0.0,Planned,NaN,0.0,0.0,30000000.0,tt2012616,...,English,"Alan Ritchson, Shailene Woodley, Pablo Schreib...",Potsy Ponciroli,NaN,Chad St. John,"Gideon Yu, Chad St. John, John Friedberg, Clif...",NaN,NaN,NaN,NaN


In [62]:
df_post = df[df['status'] == 'Post Production']  #5189
n=5189
zero_count_A = (df_post['revenue'] == 0.0).sum()
print("non zero revenue", 5198-zero_count_A)

zero_count_B = (df_post['budget'] == 0.0).sum()
print("non zero budget", 5198-zero_count_B)

print("non zero imdb votes",5189-df_post['imdb_votes'].isnull().sum())

# Correct way to count rows where all conditions are true
zero_count_C = ((df_post['revenue'] == 0.0) & 
                (df_post['budget'] == 0.0) & 
                (df_post['imdb_votes'].isnull())).sum()

# Filter the rows where all conditions are true
zero_count_c_df = df_post[
                          (df_post['budget'] != 0.0)
                        ]

print(zero_count_c_df)

print(f"Number of rows where all conditions are met: {n-zero_count_C}")

df_post.head()



non zero revenue 26
non zero budget 933
non zero imdb votes 129
              id                         title  vote_average  vote_count  \
4032        8317               Tonight at Noon           0.0         0.0   
28123      43217                       Coexist           4.8         2.0   
59424      83533          Avatar: Fire and Ash           0.0         0.0   
116342    198177      Beneath the Hagia Sophia           0.0         0.0   
149367    259382                   Off the Air           0.0         0.0   
...          ...                           ...           ...         ...   
1037160  1411059  They Live Behind The Curtain           0.0         0.0   
1037314  1411285                Perpetual Love           0.0         0.0   
1037400  1411445           Pulse and Vertebrae           0.0         0.0   
1037450  1411505           Solitude in my Mind           0.0         0.0   
1037451  1411506                  Aroma-Vision           0.0         0.0   

                  statu

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,budget,imdb_id,...,spoken_languages,cast,director,director_of_photography,writers,producers,music_composer,imdb_rating,imdb_votes,poster_path
4032,8317,Tonight at Noon,0.0,0.0,Post Production,NaN,0.0,0.0,1000000.0,tt4032552,...,English,"Lauren Ambrose, Sasha Eden, Joan Chen, Ethan H...",Michael Almereyda,Scott Miller,"Jonathan Lethem, Michael Almereyda","Caldecot Chubb, Donald Rosenfeld, Emanuel Mich...",NaN,NaN,NaN,NaN
28123,43217,Coexist,4.8,2.0,Post Production,NaN,0.0,53.0,90000.0,tt1660327,...,"English, Kinyarwanda","Domitilie, Clémentine Mukaruziga, Agnes",Adam Mazo,NaN,NaN,"Suzanne Summerlin Tzuanos, Robert Koenig, Adam...",Adam Payne,8.5,10.0,NaN
34138,50500,Hotel Caledonia,0.0,0.0,Post Production,NaN,0.0,97.0,0.0,tt0913393,...,NaN,"James Cosmo, Kris Marshall, Leslie Phillips, J...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56141,79355,Arjunan Kadhali,0.0,0.0,Post Production,NaN,0.0,0.0,0.0,NaN,...,தமிழ்,"Kalaiyarasan, Lakshmi Ramakrishnan, Suzane Geo...",Parthy Baskar,NaN,"Kabilan, Parthy Baskar",Sivasakthi Pandian,NaN,NaN,NaN,/gx0I5AYlR69ffdUmynYxNZ3OAPY.jpg
59424,83533,Avatar: Fire and Ash,0.0,0.0,Post Production,2025-12-17,0.0,0.0,400000000.0,tt1757678,...,English,"Jemaine Clement, Jeremy Irwin, Brendan Cowell,...",James Cameron,Russell Carpenter,"Josh Friedman, Shane Salerno, James Cameron, R...","Jon Landau, James Cameron, Peter M. Tobyansen",Simon Franglen,NaN,NaN,/vDdFxksDMkUJjZXd00OKJfwXKre.jpg


In [51]:
df_in = df[df['status'] == 'In Production'] #5028
n=5028

zero_count_A = (df_in['revenue'] == 0.0).sum()
print("not zero revenue", n-zero_count_A)

zero_count_B = (df_in['budget'] == 0.0).sum()
print("not zero budget", n-zero_count_B)

print("not zero votes",n-df_in['imdb_votes'].isnull().sum())
#print("in",df_in.isnull().sum())
df_in.head()

not zero revenue 59
not zero budget 584
not zero votes 45


,id,title,vote_average,vote_count,status,release_date,revenue,runtime,budget,imdb_id,...,spoken_languages,cast,director,director_of_photography,writers,producers,music_composer,imdb_rating,imdb_votes,poster_path
7305,12675,The House of Covered Mirrors,0.0,0.0,In Production,NaN,0.0,0.0,0.0,tt1904981,...,NaN,"Andria Smith, Lisa Denise, Christopher Wesley ...",Christopher Wesley Moore,NaN,NaN,NaN,NaN,NaN,NaN,/3beDPdOO2i3bVaIqxaVkHqU5yH8.jpg
7822,13494,Red Sonja,0.0,0.0,In Production,NaN,0.0,0.0,0.0,tt0800175,...,English,"Veronica Ferres, Eliza Matengu, Trevor Eve, Rh...",M.J. Bassett,NaN,"Robert E. Howard, Tasha Huo","Christa Campbell, Les Weldon, Trevor Short, Co...",NaN,NaN,NaN,/l10eghFN1nQmWwMSt91MlLvZZFA.jpg
29480,44726,Don 3,0.0,0.0,In Production,NaN,0.0,0.0,0.0,NaN,...,हिन्दी,"Ranveer Singh, Kiara Advani, Sobhita Dhulipala",Farhan Akhtar,Jason West,"Gayathri, Farhan Akhtar, Pushkar, Javed Akhtar","Farhan Akhtar, Ollwyn Dsouza, Ritesh Sidhwani",NaN,NaN,NaN,/9bOikDA268fATOUUyKCdEiFxbPZ.jpg
50572,72155,Phone Police 2: Connection Lost,0.0,0.0,In Production,NaN,0.0,0.0,0.0,NaN,...,English,"Adam Shaw, Michael Lee, Nick Pritchard, Daniel...",Ben Kay-Coles,NaN,"Ben Kay-Coles, Nick Pritchard",NaN,NaN,NaN,NaN,NaN
52197,74281,Chip Button: Born to Win,0.0,0.0,In Production,NaN,0.0,0.0,0.0,NaN,...,English,"Dave Sermon, Dave Sowden, Adam Peat, Steve San...",NaN,NaN,Dave Sowden,NaN,NaN,NaN,NaN,NaN


In [49]:
movie_fg= fs.get_feature_group(
    name='tmdb_movies',
    version=1,
)

yesterday=today - timedelta(2)

batch_data= movie_fg.filter(movie_fg.timestamp >= yesterday).read()
batch_data = batch_data.sort_values(by=['id']).reset_index(drop=True)

batch_data

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (21.44s) 


,id,title,vote_average,vote_count,status,release_date,revenue,runtime,budget,imdb_id,...,cast,director,director_of_photography,writers,producers,music_composer,imdb_rating,imdb_votes,poster_path,timestamp
0,5,Four Rooms,5.900,2650.0,Released,1995-12-09,4257354.0,98.0,4000000.0,tt0113101,...,"Unruly Julie McClean, Marc Lawrence, Jennifer ...","Robert Rodriguez, Alexandre Rockwell, Allison ...","Andrzej Sekula, Rodrigo García, Phil Parmet, G...","Robert Rodriguez, Alexandre Rockwell, Allison ...","Lawrence Bender, Alexandre Rockwell, Quentin T...",Combustible Edison,6.7,113064.0,/pyCk5JgtRZwRxnXwfrvyzukaKue.jpg,2025-01-04 00:00:00+00:00
1,6,Judgment Night,6.500,333.0,Released,1993-10-15,12136938.0,109.0,21000000.0,tt0107286,...,"Stephen Dorff, Everlast, Will Zahrn, Emilio Es...",Stephen Hopkins,Peter Levy,"Jere Cunningham, Lewis Colick","Gene Levy, Marilyn Vance, Lloyd Segan",Alan Silvestri,6.6,19437.0,/3rvvpS9YPM5HB2f4HYiNiJVtdam.jpg,2025-01-04 00:00:00+00:00
2,11,Star Wars,8.200,20722.0,Released,1977-05-25,775398007.0,121.0,11000000.0,tt0076759,...,"Larry Ward, Shelagh Fraser, Alex McCrindle, Ga...",George Lucas,Gilbert Taylor,George Lucas,"Rick McCallum, Gary Kurtz, George Lucas",John Williams,8.6,1487463.0,/6FfCtAuVAW8XJjZ7eWeLibRLWTw.jpg,2025-01-04 00:00:00+00:00
3,12,Finding Nemo,7.819,19324.0,Released,2003-05-30,940335536.0,100.0,94000000.0,tt0266543,...,"Jan Rabson, Nicholas Bird, Annelise Nolting, A...",Andrew Stanton,"Jeremy Lasky, Sharon Calahan","Blake Tucker, Adam Bronstein, Ronnie del Carme...","Graham Walters, John Lasseter",Thomas Newman,8.2,1143086.0,/eHuGQ10FUzK1mdOY69wF5pGgEf5.jpg,2025-01-04 00:00:00+00:00
4,13,Forrest Gump,8.470,27611.0,Released,1994-06-23,677387716.0,142.0,55000000.0,tt0109830,...,"Brendan Shanahan, Michael McFall, Kirk Ward, B...",Robert Zemeckis,Don Burgess,"Winston Groom, Eric Roth","Steve Tisch, Wendy Finerman, Steve Starkey",Alan Silvestri,8.8,2336706.0,/arw2vcBveWOVZr6pxd9XTd1TdQa.jpg,2025-01-04 00:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25691,1363916,Aurora Teagarden Mysteries: A Lesson in Murder,6.800,6.0,Released,2024-10-03,0.0,84.0,0.0,tt33509525,...,"Kayla Heller, Sean Depner, Aiden Howard, Skyle...",Jessica Harmon,William McKnight,"Teena Booth, Charlaine Harris, David B. Grelck","Teena Booth, Charlaine Harris, Joel S. Rice, C...",Jeff Tymoschuk,6.7,284.0,/m5w7oPvKxtCFDHqmkEZ1kq8impf.jpg,2025-01-04 00:00:00+00:00
25692,1364939,Aurora Teagarden Mysteries: Death at the Diner,5.667,3.0,Released,2024-10-10,0.0,84.0,0.0,tt33771533,...,"Kurt Evans, Kimberley Sustad, Skyler Samuels, ...",Jessica Harmon,William McKnight,"Charlaine Harris, David B. Grelck","Teena Booth, Joel S. Rice, Charles Cooper, Are...",Jeff Tymoschuk,6.8,214.0,/958StP9qK9Khen8CwFD7ED8qWD9.jpg,2025-01-04 00:00:00+00:00
25693,1369240,Surveilled,6.500,11.0,Released,2024-11-15,0.0,60.0,0.0,tt33813121,...,Ronan Farrow,"Matthew O'Neill, Perri Peltz",Matthew O'Neill,"Matthew O'Neill, Ronan Farrow, Perri Peltz","Matthew O'Neill, Nancy Abraham, Unjin Lee, Ron...",Mac Quayle,6.2,535.0,/ihBBgm2Nz754KRnsqQwhuNDvFsC.jpg,2025-01-04 00:00:00+00:00
25694,1369365,The New Alfred Hitchcock Presents,0.000,0.0,Released,1985-05-05,0.0,100.0,0.0,tt0088692,...,"Lee Ving, John Huston, Tippi Hedren, Steven Ba...","Fred Walton, Steve De Jarnatt, Randa Haines, J...","Brian R.R. Hebb, Mario DiLeo","Roald Dahl, Ethel Lina White, Christopher Crow...","Alan Barnette, Stephen Cragg, Christopher Crowe","Craig Safan, Peter Bernstein, Basil Poledouris...",7.7,413.0,/dKgYRKzmPELAfQUTDoMgiRd2cqR.jpg,2025-01-04 00:00:00+00:00


In [50]:
movie_genres_fg= fs.get_feature_group(
    name='tmdb_movie_genres',
    version=1,
)

yesterday=today - timedelta(2)

genre_batch_data= movie_genres_fg.filter(movie_genres_fg.timestamp >= yesterday).read()       #Only need todays for future predictions. Makes it a dataframe
genre_batch_data = genre_batch_data.sort_values(by=['id']).reset_index(drop=True)

df_combined = pd.merge(
    genre_batch_data,
    batch_data,
    on='id',
    how='outer',  # as we want all rows
)
df_combined



Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.87s) 


,id,timestamp_x,action,adventure,animation,comedy,crime,documentary,drama,family,...,cast,director,director_of_photography,writers,producers,music_composer,imdb_rating,imdb_votes,poster_path,timestamp_y
0,5,2025-01-04 00:00:00+00:00,0,0,0,1,0,0,0,0,...,"Unruly Julie McClean, Marc Lawrence, Jennifer ...","Robert Rodriguez, Alexandre Rockwell, Allison ...","Andrzej Sekula, Rodrigo García, Phil Parmet, G...","Robert Rodriguez, Alexandre Rockwell, Allison ...","Lawrence Bender, Alexandre Rockwell, Quentin T...",Combustible Edison,6.7,113064.0,/pyCk5JgtRZwRxnXwfrvyzukaKue.jpg,2025-01-04 00:00:00+00:00
1,6,2025-01-04 00:00:00+00:00,1,0,0,0,1,0,0,0,...,"Stephen Dorff, Everlast, Will Zahrn, Emilio Es...",Stephen Hopkins,Peter Levy,"Jere Cunningham, Lewis Colick","Gene Levy, Marilyn Vance, Lloyd Segan",Alan Silvestri,6.6,19437.0,/3rvvpS9YPM5HB2f4HYiNiJVtdam.jpg,2025-01-04 00:00:00+00:00
2,11,2025-01-04 00:00:00+00:00,1,1,0,0,0,0,0,0,...,"Larry Ward, Shelagh Fraser, Alex McCrindle, Ga...",George Lucas,Gilbert Taylor,George Lucas,"Rick McCallum, Gary Kurtz, George Lucas",John Williams,8.6,1487463.0,/6FfCtAuVAW8XJjZ7eWeLibRLWTw.jpg,2025-01-04 00:00:00+00:00
3,12,2025-01-04 00:00:00+00:00,0,0,1,0,0,0,0,1,...,"Jan Rabson, Nicholas Bird, Annelise Nolting, A...",Andrew Stanton,"Jeremy Lasky, Sharon Calahan","Blake Tucker, Adam Bronstein, Ronnie del Carme...","Graham Walters, John Lasseter",Thomas Newman,8.2,1143086.0,/eHuGQ10FUzK1mdOY69wF5pGgEf5.jpg,2025-01-04 00:00:00+00:00
4,13,2025-01-04 00:00:00+00:00,0,0,0,1,0,0,1,0,...,"Brendan Shanahan, Michael McFall, Kirk Ward, B...",Robert Zemeckis,Don Burgess,"Winston Groom, Eric Roth","Steve Tisch, Wendy Finerman, Steve Starkey",Alan Silvestri,8.8,2336706.0,/arw2vcBveWOVZr6pxd9XTd1TdQa.jpg,2025-01-04 00:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25691,1363916,2025-01-04 00:00:00+00:00,0,0,0,0,0,0,0,0,...,"Kayla Heller, Sean Depner, Aiden Howard, Skyle...",Jessica Harmon,William McKnight,"Teena Booth, Charlaine Harris, David B. Grelck","Teena Booth, Charlaine Harris, Joel S. Rice, C...",Jeff Tymoschuk,6.7,284.0,/m5w7oPvKxtCFDHqmkEZ1kq8impf.jpg,2025-01-04 00:00:00+00:00
25692,1364939,2025-01-04 00:00:00+00:00,0,0,0,0,0,0,0,0,...,"Kurt Evans, Kimberley Sustad, Skyler Samuels, ...",Jessica Harmon,William McKnight,"Charlaine Harris, David B. Grelck","Teena Booth, Joel S. Rice, Charles Cooper, Are...",Jeff Tymoschuk,6.8,214.0,/958StP9qK9Khen8CwFD7ED8qWD9.jpg,2025-01-04 00:00:00+00:00
25693,1369240,2025-01-04 00:00:00+00:00,0,0,0,0,0,1,0,0,...,Ronan Farrow,"Matthew O'Neill, Perri Peltz",Matthew O'Neill,"Matthew O'Neill, Ronan Farrow, Perri Peltz","Matthew O'Neill, Nancy Abraham, Unjin Lee, Ron...",Mac Quayle,6.2,535.0,/ihBBgm2Nz754KRnsqQwhuNDvFsC.jpg,2025-01-04 00:00:00+00:00
25694,1369365,2025-01-04 00:00:00+00:00,0,0,0,0,1,0,0,0,...,"Lee Ving, John Huston, Tippi Hedren, Steven Ba...","Fred Walton, Steve De Jarnatt, Randa Haines, J...","Brian R.R. Hebb, Mario DiLeo","Roald Dahl, Ethel Lina White, Christopher Crow...","Alan Barnette, Stephen Cragg, Christopher Crowe","Craig Safan, Peter Bernstein, Basil Poledouris...",7.7,413.0,/dKgYRKzmPELAfQUTDoMgiRd2cqR.jpg,2025-01-04 00:00:00+00:00


In [ ]:
#idea 1:
#predict upcoming through using the "planned" status in the data
#might have to change the model training 

#idea 2
# using the timestamp for the data that is not in the backfill